In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Model
from keras.layers import Input, Reshape, Dot, Embedding
from keras.optimizers import Adam
from keras.regularizers import l2

In [4]:
ratings = pd.read_csv('dataset/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['userId'].values)
n_users = ratings['user'].nunique()

item_enc = LabelEncoder()
ratings['movie'] = item_enc.fit_transform(ratings['movieId'].values)
n_movies = ratings['movie'].nunique()

ratings['rating'] = ratings['rating'].values.astype(np.float32)
min_rating = min(ratings['rating'])
max_rating = max(ratings['rating'])

n_users, n_movies, min_rating, max_rating

(610, 9724, 0.5, 5.0)

In [15]:
X = ratings[['user', 'movie']].values
Y = ratings['rating'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((90752, 2), (90752,), (10084, 2), (10084,))

In [16]:
n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [17]:
def Recommend(n_users, n_movies, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal')(user)
    u = Reshape((n_factors,))(u)
    
    movie = Input(shape=(1,))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal')(movie)
    m = Reshape((n_factors,))(m)
    
    y = Dot(axes=1)([u, m])
    
    model = Model(inputs=[user, movie], outputs=y)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    
    return model

In [18]:
model = Recommend(n_users, n_movies, n_factors)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        30500       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 50)        486200      ['input_2[0][0]']                
                                                                                              

C:\Users\Xen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [19]:
results = model.fit(x=X_train_array, y=Y_train, batch_size=64, epochs=5, verbose=1, validation_data=(X_test_array, Y_test))

Epoch 1/5
1418/1418 [==============================] - 18s 11ms/step - loss: 9.4448 - val_loss: 3.2682
Epoch 2/5
1418/1418 [==============================] - 15s 11ms/step - loss: 1.9837 - val_loss: 1.6192
Epoch 3/5
1418/1418 [==============================] - 16s 11ms/step - loss: 1.0906 - val_loss: 1.3386
Epoch 4/5
1418/1418 [==============================] - 17s 12ms/step - loss: 0.8198 - val_loss: 1.2481
Epoch 5/5
1418/1418 [==============================] - 15s 11ms/step - loss: 0.6835 - val_loss: 1.2237
